In [1]:
# %pip install python-dotenv


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['PINECONE_API_KEY'] = 'ff5095-3d6b-423b-ba58-8f2d70ac2b'
pinecone_api_key = os.environ['PINECONE_API_KEY']
# pinecone_api_key ="ff509895-3d-ba58-88df2d7c2b"
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# MODEL = 'gpt-3.5-turbo'
MODEL = 'llama3.1'

In [2]:
# %pip install langchain_openai langchain_community langchain pinecone langchain_pinecone

In [3]:
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
import torch
from sentence_transformers import SentenceTransformer


# if MODEL.startswith('gpt'):
#     model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=MODEL )
#     embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    
# else:
#     model = Ollama(model=MODEL)
#     embeddings = OllamaEmbeddings(model=MODEL)


# model.invoke('Tell me a joke')
hf_token = 'hf_AtNVMFSdzSfxraE'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Select the model based on the provided name
if MODEL.startswith('gpt'):
    model = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model_name=MODEL)
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
else:
    # Load Ollama model to GPU (if possible)
    model = Ollama(model=MODEL)
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    # embeddings = OllamaEmbeddings(model=MODEL).to(device)
    embeddings = SentenceTransformer(model_name,hf_token).to("cuda")
    embeddings.max_seq_length = 512  # or adjust based on memory availability

d:\Anaconda\envs\c_GenAI\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange



Using device: cuda


d:\Anaconda\envs\c_GenAI\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# from huggingface_hub import login

# # Replace 'your_token' with your actual Hugging Face API token
# login(token='hf_AtNVMFSd',add_to_git_credential=True)

In [5]:
import re

# Function to split text contextually
def split_text_contextually(text, split_by='sections'):
    sections = []
    if split_by == 'sections':
        # Split text by headings or specific patterns that indicate sections
        sections = re.split(r'\n\s*\n', text)  # Split by double newlines, adjust if needed
    elif split_by == 'paragraphs':
        # Split text by paragraphs
        sections = text.split('\n\n')  # Assuming paragraphs are separated by double newlines
    return sections

# Define a Document class
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata if metadata else {}

In [6]:
from langchain_community.document_loaders import PyPDFLoader


split_by = 'sections'  # Can be 'sections' or 'paragraphs'
split_texts = []

loader = PyPDFLoader("Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting security flaws-Wiley (2011).pdf")
pages = loader.load_and_split()

print("Number of pages:", len(pages))

for i, page in enumerate(pages):
    # Access text content using 'page_content'
    text = page.page_content
    
    # Perform contextual splitting
    chunks = split_text_contextually(text, split_by)
    
    # Wrap chunks in Document objects
    for chunk in chunks:
        split_texts.append(Document(page_content=chunk, metadata=page.metadata))
    
    print(f"Page {i + 1} metadata:")
    for key, value in page.metadata.items():
        print(f"  {key}: {value[:100] if isinstance(value, str) else value}")
    print("---")

Number of pages: 913
Page 1 metadata:
  source: Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting securi
  page: 1
---
Page 2 metadata:
  source: Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting securi
  page: 2
---
Page 3 metadata:
  source: Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting securi
  page: 3
---
Page 4 metadata:
  source: Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting securi
  page: 4
---
Page 5 metadata:
  source: Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting securi
  page: 5
---
Page 6 metadata:
  source: Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting securi
  page: 6
---
Page 7 metadata:
  source: Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting securi

In [7]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("ilovepdf_split2\Dafydd Stuttard, Marcus Pinto - The web application hacker's handbook_ finding and exploiting security flaws-Wiley (2011)-2.pdf")
# pages = loader.load_and_split()

# print("Number of pages:", len(pages))

# for i, page in enumerate(pages):
#     print(f"Page {i + 1} metadata:")
#     for key, value in page.metadata.items():
#         print(f"  {key}: {value[:100] if isinstance(value, str) else value}")
#     print("---")


In [ ]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce GTX 1650
Using device: cuda


In [ ]:
from langchain.prompts import PromptTemplate

template = '''
Answer the questions based on the context from the given data. While answering, specify the exact sections for the reference. If you don't know the answer, just say that you don't know.


context: {context}
question: {question}
'''

prompt = PromptTemplate(template=template)
prompt.format(context = 'Here is some context', question='here is a question')


"\nAnswer the questions based on the context from the given data. While answering, specify the exact sections for the reference. If you don't know the answer, just say that you don't know.\n\n\ncontext: Here is some context\nquestion: here is a question\n"

In [ ]:
print(type(pages))
print(len(pages))
print(type(pages[0]))

<class 'list'>
913
<class 'langchain_core.documents.base.Document'>


In [ ]:
# %pip install sentence-transformers
# %pip install tf-keras


In [12]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

from langchain_pinecone import PineconeEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
import time


pc = Pinecone(api_key=pinecone_api_key)

namespace = "RAG_Chatbot_vectors"

'''
index is a structure where your embeddings (vector representations of your data) 
are stored and managed, enabling you to efficiently search for similar vectors based on queries.
'''

index_name = "docs-quickstart-index"
embedding_dim = 384

if index_name in pc.list_indexes().names():
    pc.delete_index(name=index_name)
# to ensure that any previous state of the index is wiped clean
pc.create_index(
    name=index_name,
    dimension=embedding_dim,
    metric="cosine",      # cosinee similarity
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
)

#precautionary step to ensure that the index exists before proceeding with further operations
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(        #ServerlessSpec is used to specify the cloud provider and region for the index.
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

from langchain.embeddings import HuggingFaceEmbeddings


model_name = "sentence-transformers/all-MiniLM-L6-v2"   # used for converting embedding text into dense vector spaces
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# embeddings = PineconeEmbeddings(  
#     model=model_name,  
#     pinecone_api_key=pinecone_api_key
# )  
docsearch = PineconeVectorStore.from_documents(
    documents=split_texts,
    index_name=index_name,
    embedding=embeddings, 
    namespace=namespace 
)

time.sleep(1)

d:\Anaconda\envs\c_GenAI\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\Anaconda\envs\c_GenAI\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\Anaconda\envs\c_GenAI\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention.

In [13]:
index = pc.Index(index_name)
# retrieving and printing out information about vectors stored in the Pinecone index
for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0], 
        namespace=namespace, 
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '001ef104-28fb-42f8-8b28-641c0f2c94d6',
              'metadata': {'page': 153.0,
                           'source': 'Dafydd Stuttard, Marcus Pinto - The web '
                                     "application hacker's handbook_ finding "
                                     'and exploiting security flaws-Wiley '
                                     '(2011).pdf',
                           'text': 'Stuttard   c05.indd   V3 - 07/22/2011 Page '
                                   '118\n'
                                   '1 18 Chapter 5 n Bypassing Client-Side '
                                   'Controls\n'
                                   'Transmitting Data Via the Client\n'
                                   'It is common to see an application passing '
                                   'data to the client in a form that the \n'
                                   'end user cannot directly see or modify, '
                                   'with the expectati

In [14]:
from langchain.chains import RetrievalQA  

#creates a vector store from an existing Pinecone index
knowledge = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    namespace=namespace,    
    embedding=embeddings
)
# creates a retriever using the vector store
qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",     #All retrieved documents are combined into one and passed to the model in a single step.
    retriever=knowledge.as_retriever()
)

In [15]:
# %pip install streamlit

In [16]:
# import streamlit as st

# st.title("RAG Chatbot Application")
# st.write("Ask questions based on the provided context")

# conversation_history = ""

# query = st.text_input("Enter your question:")
# if st.button("Ask"):
#     if query:
#         # Append the new query to the conversation history
#         conversation_history += f"User: {query}\n"

#         # Combine the conversation history into a single input
#         full_query = f"{conversation_history}"

#         # Invoke the RAG model with the combined conversation history
#         response = qa.invoke({"query": full_query}).get("result")

#         # Append the model's response to the conversation history
#         conversation_history += f"Bot: {response}\n"

#         # Display the response
#         st.write(f"**Bot:** {response}")

In [18]:
conversation_history = ""

while True:
    query = input("Enter your question: ")

    # Append the new query to the conversation history
    conversation_history += f"User: {query}\n"

    # Combine the conversation history into a single input
    full_query = f"{conversation_history}"

    # Invoke the RAG model with the combined conversation history
    response = qa.invoke({"query": full_query}).get("result")

    # Append the model's response to the conversation history
    conversation_history += f"Bot: {response}\n"

    print(f"Bot: {response}")

    # Optional: Save the conversation history to a file for later use


    


Bot: Handling User Access
Authentication
Session Management
Access Control
Handling User Input
Varieties of Input
Approaches to Input Handling
Boundary Validation
Multistep Validation and Canonicalization
Handling Attackers
Handling Errors
Maintaining Audit Logs
Alerting Administrators
Reacting to Attacks 

(from the table of contents)
Bot: Based on the provided context, here are the chapter names:

1. Chapter 1: Web Application (In)security
2. Chapter 2: Core Defense Mechanisms
Bot: Based on the provided table of contents for Chapter 2, "Core Defense Mechanisms", I'll summarize the top 5 topics as follows:

1. **Handling User Access**: This topic refers to the mechanisms used by a web application to authenticate and authorize users. It includes functions such as authentication, session management, access control, and error handling.
2. **Authentication**: Authentication is the process of verifying a user's identity before granting them access to sensitive data or functionality. Chapte

KeyboardInterrupt: 

In [ ]:
# %pip install tensorflow

  Using cached tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached tensorboard-2.17.0-py3-none-any.whl.metadata (1.6 kB)


In [ ]:
# %pip install tensorflow-gpu


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "d:\Anaconda\envs\c_GenAI\Lib\site-packages\packaging\requirements.py", line 36, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\Anaconda\envs\c_GenAI\Lib\site-packages\packaging\_parser.py", line 62, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\Anaconda\envs\c_GenAI\Lib\site-packages\packaging\_parser.py", line 80, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\Anaconda\envs\c_GenAI\Lib\site-packages\packaging\_parser.py"

In [ ]:
import tensorflow as tf

# Check available GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Example TensorFlow operation to test GPU usage
if tf.config.list_physical_devices('GPU'):
    print("Using GPU")
    # Perform a simple TensorFlow operation to test GPU
    with tf.device('/GPU:0'):
        a = tf.random.normal([1000, 1000])
        b = tf.random.normal([1000, 1000])
        c = tf.matmul(a, b)
        print(c)
else:
    print("Using CPU")


Num GPUs Available:  0
Using CPU


In [ ]:
import tensorflow as tf

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is not available
